In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from scipy import stats

# ----------------------------------------------------------------
# 설정 (경로 수정 필요)
# ----------------------------------------------------------------
DATA_ROOT = './data/train'  # 데이터셋 경로
REAL_PATH = os.path.join(DATA_ROOT, 'Real')
FAKE_PATH = os.path.join(DATA_ROOT, 'Fake')
SAMPLE_SIZE = 1000  # 분석에 사용할 샘플 수 (속도를 위해 제한)

# 시각화 스타일 설정 (논문용 깔끔한 스타일)
plt.style.use('seaborn-v0_8-whitegrid')
# plt.rcParams['font.family'] = 'Times New Roman'  # 논문 폰트
plt.rcParams['font.size'] = 12

def load_images(path, limit):
    images = []
    files = glob.glob(os.path.join(path, '*.*'))[:limit]
    for f in tqdm(files, desc=f"Loading {os.path.basename(path)}"):
        img = cv2.imread(f)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 주파수 분석은 흑백으로 진행
            img = cv2.resize(img, (224, 224)) # 논문 기준 사이즈
            images.append(img)
    return np.array(images)

# ----------------------------------------------------------------
# 1. Pixel Intensity Histogram (Figure 1 구현)
# ----------------------------------------------------------------
def plot_pixel_histogram(real_imgs, fake_imgs):
    plt.figure(figsize=(10, 6))
    
    # Flatten images to 1D arrays
    real_pixels = real_imgs.flatten()
    fake_pixels = fake_imgs.flatten()
    
    # Plot KDE (Kernel Density Estimate) or Histogram
    # 데이터가 많으므로 Histogram을 부드러운 곡선처럼 보이게 bin을 많이 줌
    plt.hist(real_pixels, bins=100, density=True, color='blue', alpha=0.4, label='Real Images')
    plt.hist(fake_pixels, bins=100, density=True, color='orange', alpha=0.4, label='Diffusion Generated')
    
    plt.title('Pixel Intensity Distribution Analysis', fontsize=16, fontweight='bold')
    plt.xlabel('Pixel Intensity (0-255)')
    plt.ylabel('Density')
    plt.legend()
    plt.xlim(0, 255)
    
    plt.tight_layout()
    plt.savefig('figure1_pixel_histogram.png', dpi=300)
    print("Saved: figure1_pixel_histogram.png")
    plt.show()

# ----------------------------------------------------------------
# 2. Average Frequency Spectrum (2D DFT)
# ----------------------------------------------------------------
def get_average_spectrum(images):
    avg_spectrum = np.zeros_like(images[0], dtype=np.float64)
    
    for img in images:
        f = np.fft.fft2(img)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8) # Log scale
        avg_spectrum += magnitude_spectrum
        
    return avg_spectrum / len(images)

def plot_spectral_analysis(real_imgs, fake_imgs):
    real_spec = get_average_spectrum(real_imgs)
    fake_spec = get_average_spectrum(fake_imgs)
    
    diff_spec = real_spec - fake_spec # 차이맵 (Real - Fake)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    im1 = axes[0].imshow(real_spec, cmap='inferno')
    axes[0].set_title('Average Spectrum: Real')
    axes[0].axis('off')
    plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)
    
    im2 = axes[1].imshow(fake_spec, cmap='inferno')
    axes[1].set_title('Average Spectrum: Fake (Diffusion)')
    axes[1].axis('off')
    plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)
    
    # 차이맵: 양수(붉은색)면 Real이 더 강함, 음수(푸른색)면 Fake가 더 강함
    im3 = axes[2].imshow(diff_spec, cmap='coolwarm', vmin=-np.max(np.abs(diff_spec)), vmax=np.max(np.abs(diff_spec)))
    axes[2].set_title('Difference (Real - Fake)')
    axes[2].axis('off')
    plt.colorbar(im3, ax=axes[2], fraction=0.046, pad=0.04)
    
    plt.suptitle('Frequency Domain Analysis (Log-Magnitude)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('figure2_frequency_spectrum.png', dpi=300)
    print("Saved: figure2_frequency_spectrum.png")
    plt.show()

# ----------------------------------------------------------------
# 3. 1D Radial Profile (Power vs Frequency)
# ----------------------------------------------------------------
def get_radial_profile(spectrum):
    # 이미지 중심 계산
    y, x = np.indices(spectrum.shape)
    center = np.array(spectrum.shape) / 2
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    r = r.astype(np.int32)

    # 거리별 평균 계산 (Binning)
    tbin = np.bincount(r.ravel(), spectrum.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile

def plot_radial_profile(real_imgs, fake_imgs):
    real_spec = get_average_spectrum(real_imgs)
    fake_spec = get_average_spectrum(fake_imgs)
    
    real_profile = get_radial_profile(real_spec)
    fake_profile = get_radial_profile(fake_spec)
    
    plt.figure(figsize=(10, 6))
    
    # 고주파수 영역(오른쪽)으로 갈수록 차이가 벌어지는 것을 확인
    plt.plot(real_profile[:112], label='Real Images', color='blue', linewidth=2)
    plt.plot(fake_profile[:112], label='Fake Images', color='orange', linewidth=2, linestyle='--')
    
    plt.title('1D Power Spectrum Density (Azimuthal Average)', fontsize=16, fontweight='bold')
    plt.xlabel('Spatial Frequency (Low -> High)')
    plt.ylabel('Log Magnitude')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    plt.savefig('figure3_radial_profile.png', dpi=300)
    print("Saved: figure3_radial_profile.png")
    plt.show()

# ----------------------------------------------------------------
# 메인 실행
# ----------------------------------------------------------------
if __name__ == "__main__":
    # 데이터 로드 (경로가 맞는지 확인 후 실행하세요)
    # real_data = load_images(REAL_PATH, SAMPLE_SIZE)
    # fake_data = load_images(FAKE_PATH, SAMPLE_SIZE)
    
    # [테스트용 더미 데이터 생성] - 실제 사용 시 위 주석 해제하고 아래 3줄 주석 처리
    print("Generating Dummy Data for demo...")
    real_data = np.random.normal(128, 50, (100, 224, 224)).astype(np.uint8) # Real: 넓은 분산
    fake_data = np.random.normal(128, 20, (100, 224, 224)).astype(np.uint8) # Fake: 좁은 분산 (가우시안 블러 효과 모사 필요)

Generating Dummy Data for demo...
